## Corotines com IO-BOUND
***

Primeiro vamos resolver o codigo de forma sincrona com requests e também tipagem em python com o pydantic

In [1]:
from datetime import timedelta
from pydantic import BaseModel
import asyncio
import time
import requests
import httpx

In [2]:
class Pokemon(BaseModel): # 👈 Defines model to parse pokemon
    name: str
    types: list[str]

In [3]:
def parse_pokemon(pokemon_data: dict) -> Pokemon: # 👈 CPU-bound operation
    print(f"🔄 Parsing pokemon '{pokemon_data['name']}'")

    poke_types = []
    for poke_type in pokemon_data["types"]:
        poke_types.append(poke_type["type"]["name"])

    return Pokemon(name=pokemon_data['name'], types=poke_types)

In [4]:
def get_pokemon(name: str) -> dict | None: # 👈 IO-bound operation
    with requests.Session() as session:
        print(f"🔍 Querying for '{name}'")
        resp = session.get(f"https://pokeapi.co/api/v2/pokemon/{name}")
        print(f"🙌 Got data for '{name}'")

        try:
            resp.raise_for_status()
        except Exception as err:
            if err.response.status_code == 404:
                return None
            raise
        else:
            return resp.json()

In [5]:
def get_all(*names: str): # 👈 Sync
    started_at = time.time()
    
    for name in names: # 👈 Iterates over all names
        if data := get_pokemon(name): # 👈 Invokes async function
            result_time = round(time.time() - started_at, 2)
            pokemon = parse_pokemon(data)
            print(f"💁 {pokemon.name} is of type(s) {','.join(pokemon.types)} finish at time {result_time} seconds")
        else:
            print(f"❌ No data found for '{name}'")

    finished_at = time.time()
    elapsed_time = finished_at - started_at
    print(f"⏲️ Done in {timedelta(seconds=elapsed_time)}")

In [6]:
POKE_NAMES = [
    "blaziken", "pikachu", "lugia", "bad_name", "charizard", "venossauro", "blastoise", "butterfree",
    "beedrill", "pidgeot", "raticate", "fearow", "arbok", "raichu", "sandslash", "nidoqueen", "nidoking",
    "clefable", "ninetales", "wigglytuff"
]

In [7]:
get_all(*POKE_NAMES)

🔍 Querying for 'blaziken'
🙌 Got data for 'blaziken'
🔄 Parsing pokemon 'blaziken'
💁 blaziken is of type(s) fire,fighting finish at time 0.08 seconds
🔍 Querying for 'pikachu'
🙌 Got data for 'pikachu'
🔄 Parsing pokemon 'pikachu'
💁 pikachu is of type(s) electric finish at time 0.14 seconds
🔍 Querying for 'lugia'
🙌 Got data for 'lugia'
🔄 Parsing pokemon 'lugia'
💁 lugia is of type(s) psychic,flying finish at time 0.21 seconds
🔍 Querying for 'bad_name'
🙌 Got data for 'bad_name'
❌ No data found for 'bad_name'
🔍 Querying for 'charizard'
🙌 Got data for 'charizard'
🔄 Parsing pokemon 'charizard'
💁 charizard is of type(s) fire,flying finish at time 0.34 seconds
🔍 Querying for 'venossauro'
🙌 Got data for 'venossauro'
❌ No data found for 'venossauro'
🔍 Querying for 'blastoise'
🙌 Got data for 'blastoise'
🔄 Parsing pokemon 'blastoise'
💁 blastoise is of type(s) water finish at time 1.53 seconds
🔍 Querying for 'butterfree'
🙌 Got data for 'butterfree'
🔄 Parsing pokemon 'butterfree'
💁 butterfree is of type

Vamos utilizar o httpx para fazer requisições assincronas

In [8]:
async def get_pokemon(name: str) -> dict | None: # 👈 IO-bound operation
    async with httpx.AsyncClient() as client:
        print(f"🔍 Querying for '{name}'")
        resp = await client.get(f"https://pokeapi.co/api/v2/pokemon/{name}")
        print(f"🙌 Got data for '{name}'")

        try:
            resp.raise_for_status()
        except httpx.HTTPStatusError as err:
            if err.response.status_code == 404:
                return None
            raise
        else:
            return resp.json()

In [9]:
async def get_all(*names: str): # 👈 Async
    started_at = time.time()
    
    # 👇 Create tasks, so we start requesting all of them concurrently
    tasks = [asyncio.create_task(get_pokemon(name)) for name in names]
    
    # 👇 Await ALL
    results = await asyncio.gather(*tasks)

    for result in results:
        if result:
            result_time = round(time.time() - started_at, 2)
            pokemon = parse_pokemon(result)
            print(f"💁 {pokemon.name} is of type(s) {','.join(pokemon.types)} finish at time {result_time} seconds")
        else:
            print(f"❌ No data found for...")

    finished_at = time.time()
    elapsed_time = finished_at - started_at
    print(f"⏲️ Done in {timedelta(seconds=elapsed_time)}")

In [10]:
await get_all(*POKE_NAMES)

🔍 Querying for 'blaziken'
🔍 Querying for 'pikachu'
🔍 Querying for 'lugia'
🔍 Querying for 'bad_name'
🔍 Querying for 'charizard'
🔍 Querying for 'venossauro'
🔍 Querying for 'blastoise'
🔍 Querying for 'butterfree'
🔍 Querying for 'beedrill'
🔍 Querying for 'pidgeot'
🔍 Querying for 'raticate'
🔍 Querying for 'fearow'
🔍 Querying for 'arbok'
🔍 Querying for 'raichu'
🔍 Querying for 'sandslash'
🔍 Querying for 'nidoqueen'
🔍 Querying for 'nidoking'
🔍 Querying for 'clefable'
🔍 Querying for 'ninetales'
🔍 Querying for 'wigglytuff'
🙌 Got data for 'raticate'
🙌 Got data for 'nidoking'
🙌 Got data for 'blastoise'
🙌 Got data for 'arbok'
🙌 Got data for 'lugia'
🙌 Got data for 'blaziken'
🙌 Got data for 'bad_name'
🙌 Got data for 'pidgeot'
🙌 Got data for 'venossauro'
🙌 Got data for 'nidoqueen'
🙌 Got data for 'ninetales'
🙌 Got data for 'charizard'
🙌 Got data for 'clefable'
🙌 Got data for 'wigglytuff'
🙌 Got data for 'butterfree'
🙌 Got data for 'raichu'
🙌 Got data for 'beedrill'
🙌 Got data for 'pikachu'
🙌 Got data fo

***
### RACE
***

Vamos pegar o cenários que queremos executar o parser assim que a primeira requisição for finalizada, com isso podemos finalizar as outras em seguida ou cancelar as que não foram finalizadas, isso pode ser bem útil quando você dispara vários captchas para quebrar e gostaria de executar o resto do codigo assim que o primeiro captcha finalizar.

In [11]:
async def get_all(*names: str):
    started_at = time.time()

    tasks = [asyncio.create_task(get_pokemon(name)) for name in names]

    # 👇 Process the tasks individually as they become available
    for corotine in asyncio.as_completed(tasks):
        result = await corotine # 👈 You still need to await

        if result:
            result_time = round(time.time() - started_at, 2)
            pokemon = parse_pokemon(result)
            print(f"💁 {pokemon.name} is of type(s) {','.join(pokemon.types)} finish at time {result_time} seconds")
        else:
            print(f"❌ No data found for...")
    
    finished_at = time.time()
    elapsed_time = finished_at - started_at
    print(f"⏲️ Done in {timedelta(seconds=elapsed_time)}")

In [12]:
await get_all(*POKE_NAMES)

🔍 Querying for 'blaziken'
🔍 Querying for 'pikachu'
🔍 Querying for 'lugia'
🔍 Querying for 'bad_name'
🔍 Querying for 'charizard'
🔍 Querying for 'venossauro'
🔍 Querying for 'blastoise'
🔍 Querying for 'butterfree'
🔍 Querying for 'beedrill'
🔍 Querying for 'pidgeot'
🔍 Querying for 'raticate'
🔍 Querying for 'fearow'
🔍 Querying for 'arbok'
🔍 Querying for 'raichu'
🔍 Querying for 'sandslash'
🔍 Querying for 'nidoqueen'
🔍 Querying for 'nidoking'
🔍 Querying for 'clefable'
🔍 Querying for 'ninetales'
🔍 Querying for 'wigglytuff'
🙌 Got data for 'blastoise'
🔄 Parsing pokemon 'blastoise'
💁 blastoise is of type(s) water finish at time 0.37 seconds
🙌 Got data for 'bad_name'
❌ No data found for...
🙌 Got data for 'pikachu'
🙌 Got data for 'butterfree'
🔄 Parsing pokemon 'pikachu'
💁 pikachu is of type(s) electric finish at time 0.39 seconds
🙌 Got data for 'beedrill'
🙌 Got data for 'raticate'
🔄 Parsing pokemon 'butterfree'
💁 butterfree is of type(s) bug,flying finish at time 0.4 seconds
🙌 Got data for 'wigglytuf